In [8]:
import numpy as np
import tensorflow as tf
import os
import cv2
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load images and labels from the dataset
def load_data_from_directory(image_dir):
    images = []
    labels = []

    for label in os.listdir(image_dir):
        label_dir = os.path.join(image_dir, label)
        if os.path.isdir(label_dir):
            for filename in os.listdir(label_dir):
                image_path = os.path.join(label_dir, filename)
                image = cv2.imread(image_path)
                if image is not None:
                    image = cv2.resize(image, (128, 128))  # Resize to match training size
                    images.append(image)
                    labels.append(label)

    return images, labels

# Define dataset path
base_path = os.getcwd()  # Current directory
image_dir = os.path.join(base_path, "../data/train")  # Adjust as needed

# Load dataset
images, labels = load_data_from_directory(image_dir)

# Convert images to NumPy array and normalize
images = np.array(images) / 255.0

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
y = to_categorical(encoded_labels)  # One-hot encode labels

# Split into test set (assuming 20% was used before)
_, X_test, _, y_test = train_test_split(images, y, test_size=0.2, random_state=42)

# Load the trained model
model_path = os.path.join(base_path, "../models/detection_model.keras")
model = tf.keras.models.load_model(model_path)

# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels
y_true = np.argmax(y_test, axis=1)  # Convert one-hot encoding to class labels

# Get class names
class_names = label_encoder.classes_

# 1. **Classification Report**
print("\n=== Classification Report ===")
print(classification_report(y_true, y_pred_classes, target_names=class_names))

# 2. **Confusion Matrix**
conf_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# 3. **AUC-ROC Scores**
auc_scores = roc_auc_score(y_test, y_pred, average=None)

print("\n=== AUC Scores for Each Class ===")
for class_name, auc in zip(class_names, auc_scores):
    print(f"AUC for {class_name}: {auc:.4f}")

# Plot ROC Curve
plt.figure(figsize=(8, 6))
for i, class_name in enumerate(class_names):
    fpr, tpr, _ = roc_curve(y_test[:, i], y_pred[:, i])
    plt.plot(fpr, tpr, label=f"{class_name} (AUC = {auc_scores[i]:.2f})")

plt.plot([0, 1], [0, 1], 'k--')  # Diagonal reference line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves for Multi-Class Classification")
plt.legend()
plt.show()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 589ms/step

=== Classification Report ===


ValueError: Number of classes, 2, does not match size of target_names, 1. Try specifying the labels parameter